In [ ]:
import geopandas
import geoviews as gv
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import panel as pn
import stmtools
import xarray as xr

from dask.distributed import Client
from holoviews import opts
from holoviews import streams

In [ ]:
# hv configurations
hv.extension('bokeh')
opts.defaults(opts.Points(tools=['box_select', 'lasso_select']))
gv.output(dpi=120)

# Visualize STM & contextual data

In [ ]:
# input data paths
DATA_DIR = '/project/caroline/Public/demo_mobyle/data'

# Space-time matrix data
DATA_STEM = 'full-pixel_psi_amsterdam_tsx_asc_t116_v4_ampl_std_H_c16643'
CSV_STM_PATH = f'{DATA_DIR}/depsi_products/{DATA_STEM}.csv'
ZARR_STM_PATH = f'{DATA_STEM}.zarr'

# time-dependent variable: total precipitation
TP_DATA_PATH = f'{DATA_DIR}/ERA5/ERA5-land-monthly_2015-2023_NL.nc'
# space-dependent variable: BAG dataset for AMS
BAG_DATA_PATH = f'{DATA_DIR}/BAG/bag_light_AMS_WGS84.gpkg'

In [ ]:
! du -h $CSV_STM_PATH

## 1. Setup Dask cluster

In [ ]:
from dask.distributed import Client

client = LocalCluster(n_workers=2, threads_per_worker=4)

## 2. Convert STM data format: CSV -> Zarr

We use [STMTools](https://github.com/MotionbyLearning/stmtools/tree/main) to load the STM dataset from a CSV file and save it in Zarr format:

In [ ]:
# stm = stmtools.from_csv(CSV_STM_PATH)
# stm = stm.persist()
# stm.to_zarr(ZARR_STM_PATH, mode='w')

## 3. STM and contextual data

We consider three datasets:
* The **STM** dataset, with space- and time-dependent variables (e.g. deformation);
* ERA5-land monthly **total precipitation** data, of which we consider the only dependence on time;
* **Building footprings** from the BAG dataset (space dependence only).

In [ ]:
# STM dataset, space-time dependent
stm = xr.open_zarr(ZARR_STM_PATH)

In [ ]:
# Total precipitation, (space-)time dependent
ds = xr.open_dataset(TP_DATA_PATH)
tp = ds['tp'].sel(
    latitude=stm['lat'].mean(), 
    longitude=stm['lon'].mean(),
    time=stm.time, 
    expver=1,
    method='nearest',
)
tp = tp.assign_coords(time=stm.time)

In [ ]:
# BAG dataset, space dependent
bbox = (4.88, 52.36, 4.92, 52.38) 
bag = geopandas.read_file(BAG_DATA_PATH, bbox=bbox)

## 4. Visualizing the datasets

We create a scatter plot on a base map for the STM data points:

In [ ]:
# create points plot
xy = stm[['lat', 'lon']].to_dataframe()
points = xy.hvplot.points(
    'lon', 
    'lat', 
    geo=True, 
    color='red',
    size=1,
    tiles='ESRI', 
)

We plot for the deformation associated to each point as a function of time. The only points selected in the previous panel will be considered!

In [ ]:
TOO_MANY_POINTS = 10
VARIABLE = 'deformation'

# create stream for a selection of points
selection = streams.Selection1D(source=points)

def plot_variable(index):
    """ Plot STM variable vs time for a sub-set of points. """
    if not index or len(index) > TOO_MANY_POINTS:
        # for no or too many points, plot point 0
        return plot_variable([0])
    else:
        lines = [
            stm.isel(space=i).hvplot(x='time', y=VARIABLE)
            for i in index
        ]
        return hv.Overlay(lines)

# create interactive variable plot
deformation = hv.DynamicMap(plot_variable, streams=[selection])

The total precipitation is also plotted as a function of the time.

In [ ]:
precipitation = tp.hvplot.line(x='time', y='tp')

Finally, the building footprints are also plotted out, and colored according to the year of construction.

In [ ]:
buildings = gv.Polygons(bag, vdims=[('bouwjaar', 'Year Built')])

We set some parameters, and compose the final layout:

In [ ]:
points = points.opts(frame_width=500, frame_height=500)
buildings = buildings.opts(frame_width=500, frame_height=500, tools=['hover'])
deformation = deformation.opts(frame_width=500)
precipitation = precipitation.opts(frame_width=500)

plot = (points + buildings + deformation + precipitation).cols(2)

The plot can be visualized in the notebook:

In [ ]:
plot

.. or one can visualize it independently with a dedicated `panel` server:

In [ ]:
pn.panel(plot)